In [6]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm


class ReversePINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ReversePINN, self).__init__()
        # Define neural network layers
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        # Output layer for concentration (C)
        self.layers.append(nn.Linear(hidden_size, output_size))

        # Learnable parameters: D and Kd (initially set as trainable parameters)
        self.D = nn.Parameter(
            torch.tensor([10.0], dtype=torch.float32, requires_grad=True)
        )  # Diffusion coefficient
        self.Kd = nn.Parameter(
            torch.tensor([10.0], dtype=torch.float32, requires_grad=True)
        )  # Sorption coefficient

        # Loss function and optimizer
        self.loss = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)

    def forward(self, x):
        # Feedforward pass to get concentration profile
        for layer in self.layers:
            x = layer(x)
        return x

    def residual_loss(self, xtrain):
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)  # Predicted concentration

        # Compute gradients for the residuals
        u_x_t = torch.autograd.grad(
            u_pred,
            g,
            torch.ones([g.shape[0], 1]).to("cuda"),
            retain_graph=True,
            create_graph=True,
        )[0]
        u_xx_tt = torch.autograd.grad(
            u_x_t, g, torch.ones(g.shape).to("cuda"), create_graph=True
        )[0]

        # Diffusion term: u_xx (second spatial derivative)
        u_xx = u_xx_tt[:, [0]]

        # Physics-informed residual (diffusion-sorption-reaction equation)
        residual = u_x_t + self.D * u_xx - self.Kd * u_pred

        return self.loss(
            residual, torch.zeros_like(residual).to("cuda")
        )  # Residual should be zero

    def total_loss(self, xtrain, utrain):
        # Data loss: Compare predicted concentration to true concentration
        data_loss = self.loss(self.forward(xtrain), utrain)

        # Residual loss: Physics-informed
        residual_loss = self.residual_loss(xtrain)

        return data_loss + residual_loss

    def train_model(self, xtrain, utrain, epochs=1000):
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain)
            loss.backward()
            self.optimizer.step()
            if epoch % 100 == 0:
                print(
                    f"Epoch {epoch}, Loss {loss.item()}, D {self.D.item()}, Kd {self.Kd.item()}"
                )

In [2]:
import h5py

Nx = 100
Nt = 201

with h5py.File("diff_sorp.h5", "r") as f:
    dtrue = f["1"]["D"][()]
    ktrue = f["1"]["Kd"][()]
    C_all = f["1"]["data"][:]
    f.close()
# Create input (x, t) and output (C) tensors
x = np.linspace(0, 1, Nx)
t = np.linspace(0, 1, Nt)
X, T = np.meshgrid(x, t)
X_train = torch.tensor(
    np.vstack((X.flatten(), T.flatten())).T, dtype=torch.float32
).cuda()
C_train = torch.tensor(C_all.flatten(), dtype=torch.float32).cuda()
X_train.shape, C_train.shape

(torch.Size([20100, 2]), torch.Size([20100]))

In [3]:
dtrue, ktrue

(82.44376791378382, 92.0820718580985)

In [4]:
idx = np.random.choice(Nx * Nt, 10000, replace=False)
X_train = X_train[idx]
C_train = C_train[idx]
X_train.shape, C_train.shape

(torch.Size([10000, 2]), torch.Size([10000]))

In [7]:
model = ReversePINN(2, 20, 1).cuda()
model.train_model(X_train, C_train, epochs=10000)

  0%|          | 12/10000 [00:00<02:51, 58.37it/s]

Epoch 0, Loss 3.148857593536377, D 9.999899864196777, Kd 9.999899864196777


  1%|          | 110/10000 [00:01<02:40, 61.44it/s]

Epoch 100, Loss 0.186541348695755, D 10.0056734085083, Kd 9.994372367858887


  2%|▏         | 208/10000 [00:03<02:33, 63.69it/s]

...

Epoch 7700, Loss 0.09293389320373535, D 10.207327842712402, Kd 9.977411270141602


 78%|███████▊  | 7813/10000 [02:07<00:35, 61.83it/s]

Epoch 7800, Loss 0.09292813390493393, D 10.207804679870605, Kd 9.977029800415039


 79%|███████▉  | 7896/10000 [02:09<00:34, 61.20it/s]


KeyboardInterrupt: 